In [2]:
import os
import cv2
import numpy as np
from PIL import Image
import pickle
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

# Paths to crack and non-crack image folders
crack_path = 'C:\\New folder\\Dr. Surya\\Project\\crack'
noncrack_path = 'C:\\New folder\\Dr. Surya\\Project\\noncrack'

# Define target size
TARGET_SIZE = (224, 224) 

# Load all image files and annotations  
crack_images = [(os.path.join(crack_path, f), 1) for f in os.listdir(crack_path)] 
noncrack_images = [(os.path.join(noncrack_path, f), 0) for f in os.listdir(noncrack_path)]

images = crack_images + noncrack_images
np.random.shuffle(images)

# Preprocess and extract features  
def preprocess(image):  
    img = Image.open(image)
    img = img.resize(TARGET_SIZE)  
    x = np.array(img)/255

    return x, label

X = []
y = []

for img, label in images:
    x, label = preprocess(img)
  
    X.append(x)
    y.append(label)

# Split train/test  
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Resize images to a consistent size
resized_images_train = []
for img in X_train:
    resized_img = cv2.resize(img, (224, 224))
    resized_img = cv2.convertScaleAbs(resized_img) # Convert to CV_8U format
    resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB) # Convert to RGB format
    resized_images_train.append(resized_img)

resized_images_test= []
for img in X_test:
    resized_img = cv2.resize(img, (224, 224))
    resized_img= cv2.convertScaleAbs(resized_img) # Convert to CV_8U format
    resized_img= cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB) # Convert to RGB format
    resized_images_test.append(resized_img)

# Convert resized_images to NumPy array
X_train = np.empty((len(resized_images_train), 224, 224, 3), dtype=np.float32)
for i, img in enumerate(resized_images_train):
    X_train[i] = img

X_test = np.empty((len(resized_images_test), 224, 224, 3), dtype=np.float32)
for i, img in enumerate(resized_images_test):
    X_test[i] = img

y_train = np.array(y_train)
y_test = np.array(y_test)

# Define model
model = Sequential()

# Convolutional layer 1
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3))) 

# Max pooling layer 1 
model.add(MaxPooling2D((2,2)))

# Convolutional layer 2
model.add(Conv2D(64, (3,3), activation='relu'))

# Max pooling layer 2
model.add(MaxPooling2D((2,2))) 

# Convolutional layer 3  
model.add(Conv2D(128, (3,3), activation='relu'))

# Max pooling layer 3
model.add(MaxPooling2D((2,2)))

# Flatten layer
model.add(Flatten())

# Dense layer  
model.add(Dense(256, activation='relu'))

# Output layer with sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(X_test, y_test))

# Evaluate model on test set
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test loss: ', test_loss)
print('Test accuracy: ', test_acc)

# Save the model 
model.save('C:\\New folder\\Dr. Surya\\Project\\CNN1.h5')

Epoch 1/10
47/47 [==============================] - 84s 2s/step - loss: 0.2651 - accuracy: 0.8920 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 2/10
47/47 [==============================] - 83s 2s/step - loss: 2.0744e-04 - accuracy: 1.0000 - val_loss: 3.9083e-05 - val_accuracy: 1.0000
Epoch 3/10
47/47 [==============================] - 72s 2s/step - loss: 2.9184e-05 - accuracy: 1.0000 - val_loss: 2.8650e-05 - val_accuracy: 1.0000
Epoch 4/10
47/47 [==============================] - 73s 2s/step - loss: 2.2925e-05 - accuracy: 1.0000 - val_loss: 2.2839e-05 - val_accuracy: 1.0000
Epoch 5/10
47/47 [==============================] - 74s 2s/step - loss: 1.8490e-05 - accuracy: 1.0000 - val_loss: 1.8432e-05 - val_accuracy: 1.0000
Epoch 6/10
47/47 [==============================] - 74s 2s/step - loss: 1.5136e-05 - accuracy: 1.0000 - val_loss: 1.5247e-05 - val_accuracy: 1.0000
Epoch 7/10
47/47 [==============================] - 74s 2s/step - loss: 1.2643e-05 - accuracy: 1.0000 - val_loss: 1.2729

C:\Users\peima\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
